In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import pandas as pd
import numpy as np
import librosa
import torch
import os

In [23]:
here = os.getcwd()

In [24]:
file_dir = os.path.join(here, "/kaggle/input/data-for-cpm/train/train")
file_dir

'/kaggle/input/data-for-cpm/train/train'

In [25]:
df = pd.read_csv('/kaggle/input/data-for-cpm/AI Challenge Ground Truth.csv', header=None)
data = dict(zip(df[0], df[1]))

In [27]:
import librosa
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm

# Функция для извлечения признаков из аудио файла
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    # Извлечение MFCC признаков
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = np.mean(mfccs.T, axis=0)  # Средние значения MFCC по времени
    return mfccs_mean

# Пример загрузки аудио файлов и извлечения признаков
file_paths = list(data.keys())
labels = [data[i] for i in file_paths] # Метки классов для аудио файлов

features = []
for fp in tqdm(file_paths):
    features += [extract_features(os.path.join(file_dir, fp))]

# max_len = len(max(features, key=lambda x: len(x)))
# print(max_len)

# features = [(list(item) + [item.mean()] * (max_len - len(item)))[:800] for item in tqdm(features)]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Масштабирование признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Обучение модели логистической регрессии
model = CatBoostClassifier(iterations=1000, learning_rate=0.001, depth=6, task_type='GPU', verbose=100)
model.fit(X_train, y_train)

# Оценка модели
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

  0%|          | 0/8803 [00:00<?, ?it/s]

0:	learn: 0.6930560	total: 37.7ms	remaining: 37.7s
100:	learn: 0.6848701	total: 3.38s	remaining: 30s
200:	learn: 0.6780264	total: 6.77s	remaining: 26.9s
300:	learn: 0.6722127	total: 10.1s	remaining: 23.6s
400:	learn: 0.6673307	total: 13.6s	remaining: 20.2s
500:	learn: 0.6630754	total: 17s	remaining: 16.9s
600:	learn: 0.6595013	total: 20.3s	remaining: 13.5s
700:	learn: 0.6563625	total: 23.7s	remaining: 10.1s
800:	learn: 0.6536072	total: 27.1s	remaining: 6.73s
900:	learn: 0.6512553	total: 30.4s	remaining: 3.34s
999:	learn: 0.6491497	total: 33.8s	remaining: 0us
Accuracy: 0.65
